# Estructura del Transformer Mini

🔧 1. Embedding + Positional Encodin

In [ ]:
import torch  # Importa PyTorch, una biblioteca para computación numérica y aprendizaje profundo
import torch.nn as nn  # Importa el módulo de redes neuronales de PyTorch
import math  # Importa la biblioteca matemática estándar de Python

class PositionalEncoding(nn.Module):  # Define una clase para codificación posicional, hereda de nn.Module
    def __init__(self, d_model, max_len=5000):  # Constructor, inicializa los parámetros del modelo
        super().__init__()  # Llama al constructor de la clase base
        pe = torch.zeros(max_len, d_model)  # Crea un tensor de ceros para almacenar las codificaciones posicionales
        position = torch.arange(0, max_len).unsqueeze(1)  # Genera índices de posición y añade una dimensión
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))  # Calcula el término de división para las frecuencias

        pe[:, 0::2] = torch.sin(position * div_term)  # Aplica la función seno a las posiciones pares
        pe[:, 1::2] = torch.cos(position * div_term)  # Aplica la función coseno a las posiciones impares

        self.pe = pe.unsqueeze(0)  # Añade una dimensión para el batch

    def forward(self, x):  # Método forward, aplica la codificación posicional a la entrada
        x = x + self.pe[:, :x.size(1)].to(x.device)  # Suma las codificaciones posicionales a la entrada
        return x  # Devuelve la entrada modificada


⚡ 2. Self-Attention

In [ ]:
class SelfAttention(nn.Module):  # Define una clase para la atención propia, hereda de nn.Module
    def __init__(self, embed_dim):  # Constructor, inicializa los parámetros del modelo
        super().__init__()  # Llama al constructor de la clase base
        self.embed_dim = embed_dim  # Dimensión del embedding

        self.qkv = nn.Linear(embed_dim, embed_dim * 3)  # Proyección lineal para queries, keys y values
        self.out = nn.Linear(embed_dim, embed_dim)  # Proyección lineal para la salida

    def forward(self, x):  # Método forward, aplica la atención propia
        B, T, C = x.shape  # Obtiene las dimensiones del batch, secuencia y embedding
        qkv = self.qkv(x)  # Calcula queries, keys y values concatenados
        q, k, v = qkv.chunk(3, dim=-1)  # Divide qkv en tres partes: queries, keys y values

        scores = q @ k.transpose(-2, -1) / math.sqrt(C)  # Calcula las puntuaciones de atención
        weights = scores.softmax(dim=-1)  # Aplica softmax para obtener los pesos de atención
        attended = weights @ v  # Calcula la salida atendida

        return self.out(attended)  # Devuelve la salida proyectada


🎩 3. Multi-Head Attention

In [ ]:
class MultiHeadAttention(nn.Module):  # Define una clase para la atención multi-cabeza, hereda de nn.Module
    def __init__(self, embed_dim, num_heads):  # Constructor, inicializa los parámetros del modelo
        super().__init__()  # Llama al constructor de la clase base
        assert embed_dim % num_heads == 0  # Verifica que embed_dim sea divisible por num_heads
        self.head_dim = embed_dim // num_heads  # Calcula la dimensión de cada cabeza
        self.num_heads = num_heads  # Número de cabezas

        self.qkv = nn.Linear(embed_dim, embed_dim * 3)  # Proyección lineal para queries, keys y values
        self.out = nn.Linear(embed_dim, embed_dim)  # Proyección lineal para la salida

    def forward(self, x):  # Método forward, aplica la atención multi-cabeza
        B, T, C = x.shape  # Obtiene las dimensiones del batch, secuencia y embedding
        qkv = self.qkv(x).view(B, T, 3, self.num_heads, self.head_dim).permute(2, 0, 3, 1, 4)  # Calcula y reorganiza qkv
        q, k, v = qkv[0], qkv[1], qkv[2]  # Divide qkv en queries, keys y values

        scores = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)  # Calcula las puntuaciones de atención
        weights = scores.softmax(dim=-1)  # Aplica softmax para obtener los pesos de atención
        context = weights @ v  # Calcula el contexto atendido

        out = context.transpose(1, 2).contiguous().view(B, T, C)  # Reorganiza y combina las cabezas
        return self.out(out)  # Devuelve la salida proyectada


🧱 4. Bloque Transformer

In [ ]:
class TransformerBlock(nn.Module):  # Define un bloque Transformer, hereda de nn.Module
    def __init__(self, embed_dim, num_heads, ff_hidden_dim):  # Constructor, inicializa los parámetros del modelo
        super().__init__()  # Llama al constructor de la clase base
        self.attn = MultiHeadAttention(embed_dim, num_heads)  # Crea una capa de atención multi-cabeza
        self.norm1 = nn.LayerNorm(embed_dim)  # Normalización por capas para la salida de atención
        self.ff = nn.Sequential(  # Crea una red feed-forward
            nn.Linear(embed_dim, ff_hidden_dim),  # Proyección lineal hacia una dimensión oculta
            nn.ReLU(),  # Función de activación ReLU
            nn.Linear(ff_hidden_dim, embed_dim)  # Proyección lineal de vuelta a la dimensión original
        )
        self.norm2 = nn.LayerNorm(embed_dim)  # Normalización por capas para la salida de la red feed-forward

    def forward(self, x):  # Método forward, aplica el bloque Transformer
        x = x + self.attn(self.norm1(x))  # Aplica atención multi-cabeza con residual
        x = x + self.ff(self.norm2(x))  # Aplica red feed-forward con residual
        return x  # Devuelve la salida del bloque


🧠 5. Mini GPT (Tokenizador muy simple)

In [ ]:
class MiniTransformerModel(nn.Module):  # Define un modelo Transformer mini, hereda de nn.Module
    def __init__(self, vocab_size, embed_dim, max_len, num_heads, ff_hidden_dim, num_layers):  # Constructor
        super().__init__()  # Llama al constructor de la clase base
        self.token_embed = nn.Embedding(vocab_size, embed_dim)  # Embedding para los tokens
        self.pos_embed = PositionalEncoding(embed_dim, max_len)  # Codificación posicional
        self.transformer_blocks = nn.Sequential(*[  # Secuencia de bloques Transformer
            TransformerBlock(embed_dim, num_heads, ff_hidden_dim)
            for _ in range(num_layers)  # Repite el bloque num_layers veces
        ])
        self.lm_head = nn.Linear(embed_dim, vocab_size)  # Proyección lineal para la salida del modelo

    def forward(self, x):  # Método forward, aplica el modelo completo
        x = self.token_embed(x)  # Aplica el embedding de tokens
        x = self.pos_embed(x)  # Aplica la codificación posicional
        x = self.transformer_blocks(x)  # Pasa por los bloques Transformer
        return self.lm_head(x)  # Devuelve la salida proyectada (sin softmax)


In [ ]:
# Simulamos datos
vocab_size = 100
seq_len = 32
batch_size = 8
embed_dim = 64

model = MiniTransformerModel(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    max_len=seq_len,
    num_heads=4,
    ff_hidden_dim=128,
    num_layers=2
)

dummy_input = torch.randint(0, vocab_size, (batch_size, seq_len))
output = model(dummy_input)  # (batch_size, seq_len, vocab_size)
print(output.shape)  # Debe ser (8, 32, 100)


torch.Size([8, 32, 100])


In [8]:
class SimpleTokenizer:
    def __init__(self, corpus):
        chars = sorted(list(set(corpus)))
        self.stoi = {ch: i for i, ch in enumerate(chars)}
        self.itos = {i: ch for ch, i in self.stoi.items()}
        self.vocab_size = len(self.stoi)

    def encode(self, text):
        return [self.stoi[c] for c in text]

    def decode(self, tokens):
        return ''.join([self.itos[i] for i in tokens])


In [9]:
text = "hola mundo"
tokenizer = SimpleTokenizer(text)
ids = tokenizer.encode("hola")
print(ids)  # → lista de números
print(tokenizer.decode(ids))  # → "hola"


[3, 7, 4, 1]
hola
